In [31]:
# This script is for DNN socio variables
# Take Dataset3 NYC for example

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch

# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [33]:
X_socio_test_passed_1 = pd.read_csv("../Data/01 Processed Dataset/Socio_NYC.csv") #input
X_sociodemo_test_passed = X_socio_test_passed_1.set_index('GEOID')
y = X_sociodemo_test_passed.pop('obesity_cr')

In [34]:
len(X_sociodemo_test_passed)

1995

In [35]:
# new code
# sociodemographic predictors

y_dnn_socio_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

avg_r_squared = 0

def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

for train_index, test_index in ten_fold.split(X_sociodemo_test_passed):
    print("TEST:", test_index)

    X_train, X_test = X_sociodemo_test_passed.iloc[train_index], X_sociodemo_test_passed.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #scaler = StandardScaler()
    
    #scaled_X_train = scaler.fit_transform(X_train)
    #scaled_X_test = scaler.transform(X_test)
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    
    tf.random.set_seed(42)
    dnn_model = keras.models.Sequential([
        keras.layers.Dense(176,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(208,activation="relu"),
        keras.layers.Dense(64,activation="relu"),
        keras.layers.Dense(208,activation="relu"),
        keras.layers.Dense(208,activation="relu"),
        keras.layers.Dense(1)
    ])
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    dnn_model.compile(optimizer="adam", loss=keras.losses.mean_absolute_error, metrics=[keras.metrics.mean_squared_error])
    dnn_model.fit(x=scaled_X_train, y=y_train, validation_split=0.1, epochs=50, callbacks=[early_stop], verbose=2) 
    
    this_y_predict = dnn_model.predict(scaled_X_test).flatten()

    y_dnn_socio_predict = y_dnn_socio_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [  29   49   56   65   69   70   99  109  111  120  124  128  135  211
  212  231  233  237  239  251  254  256  270  281  298  305  306  307
  316  324  342  344  350  353  361  366  367  368  382  383  394  411
  414  416  420  422  427  429  433  450  464  480  483  485  486  514
  526  527  530  532  548  552  554  561  570  581  582  583  602  614
  617  630  679  741  743  752  756  759  785  798  819  824  843  849
  881  889  892  903  915  921  926  930  932  940  949  950  988  999
 1001 1009 1052 1087 1106 1113 1116 1118 1125 1137 1160 1169 1176 1177
 1182 1216 1221 1234 1235 1242 1244 1252 1255 1268 1270 1273 1274 1284
 1285 1287 1298 1303 1305 1316 1318 1322 1341 1343 1344 1357 1359 1379
 1384 1386 1412 1428 1429 1444 1448 1469 1476 1490 1497 1506 1531 1532
 1563 1573 1593 1594 1600 1604 1616 1619 1647 1659 1668 1671 1673 1675
 1681 1684 1686 1716 1720 1726 1732 1736 1740 1758 1764 1770 1784 1810
 1813 1814 1820 1855 1856 1867 1907 1918 1926 1934 1935 1936 1940 1952


Epoch 1/50
51/51 - 0s - loss: 11.0729 - mean_squared_error: 211.1550 - val_loss: 4.1982 - val_mean_squared_error: 28.2311 - 490ms/epoch - 10ms/step
Epoch 2/50
51/51 - 0s - loss: 4.0610 - mean_squared_error: 32.1197 - val_loss: 3.2658 - val_mean_squared_error: 17.4734 - 68ms/epoch - 1ms/step
Epoch 3/50
51/51 - 0s - loss: 3.5523 - mean_squared_error: 22.6919 - val_loss: 3.5055 - val_mean_squared_error: 17.4991 - 84ms/epoch - 2ms/step
Epoch 4/50
51/51 - 0s - loss: 3.0368 - mean_squared_error: 17.6130 - val_loss: 2.9104 - val_mean_squared_error: 12.4328 - 88ms/epoch - 2ms/step
Epoch 5/50
51/51 - 0s - loss: 2.7018 - mean_squared_error: 13.3632 - val_loss: 2.5936 - val_mean_squared_error: 9.9585 - 82ms/epoch - 2ms/step
Epoch 6/50
51/51 - 0s - loss: 2.6008 - mean_squared_error: 11.9775 - val_loss: 2.4782 - val_mean_squared_error: 8.7264 - 75ms/epoch - 1ms/step
Epoch 7/50
51/51 - 0s - loss: 2.3058 - mean_squared_error: 9.0003 - val_loss: 2.0345 - val_mean_squared_error: 6.6578 - 67ms/epoch - 1

51/51 - 0s - loss: 3.6728 - mean_squared_error: 23.4453 - val_loss: 2.8393 - val_mean_squared_error: 12.5969 - 76ms/epoch - 1ms/step
Epoch 4/50
51/51 - 0s - loss: 3.1504 - mean_squared_error: 17.8966 - val_loss: 2.5566 - val_mean_squared_error: 11.3531 - 68ms/epoch - 1ms/step
Epoch 5/50
51/51 - 0s - loss: 2.7461 - mean_squared_error: 13.7627 - val_loss: 3.4558 - val_mean_squared_error: 16.1887 - 67ms/epoch - 1ms/step
Epoch 6/50
51/51 - 0s - loss: 2.5398 - mean_squared_error: 10.9261 - val_loss: 3.1876 - val_mean_squared_error: 13.1532 - 91ms/epoch - 2ms/step
Epoch 7/50
51/51 - 0s - loss: 2.2583 - mean_squared_error: 8.8837 - val_loss: 2.2743 - val_mean_squared_error: 7.4209 - 75ms/epoch - 1ms/step
Epoch 8/50
51/51 - 0s - loss: 2.1008 - mean_squared_error: 7.6204 - val_loss: 2.6397 - val_mean_squared_error: 9.2428 - 81ms/epoch - 2ms/step
Epoch 9/50
51/51 - 0s - loss: 1.9647 - mean_squared_error: 6.6132 - val_loss: 2.0676 - val_mean_squared_error: 5.8314 - 82ms/epoch - 2ms/step
Epoch 10/

Epoch 24/50
51/51 - 0s - loss: 1.4673 - mean_squared_error: 3.5524 - val_loss: 1.5579 - val_mean_squared_error: 3.4963 - 83ms/epoch - 2ms/step
Epoch 25/50
51/51 - 0s - loss: 1.2972 - mean_squared_error: 3.0261 - val_loss: 1.4595 - val_mean_squared_error: 3.4183 - 77ms/epoch - 2ms/step
Epoch 26/50
51/51 - 0s - loss: 1.2669 - mean_squared_error: 2.7696 - val_loss: 1.3213 - val_mean_squared_error: 2.7670 - 72ms/epoch - 1ms/step
Epoch 27/50
51/51 - 0s - loss: 1.3002 - mean_squared_error: 3.0424 - val_loss: 1.3563 - val_mean_squared_error: 3.1755 - 66ms/epoch - 1ms/step
Epoch 28/50
51/51 - 0s - loss: 1.2665 - mean_squared_error: 2.8111 - val_loss: 1.4392 - val_mean_squared_error: 3.2233 - 67ms/epoch - 1ms/step
Epoch 29/50
51/51 - 0s - loss: 1.2451 - mean_squared_error: 2.7044 - val_loss: 1.5315 - val_mean_squared_error: 3.5424 - 66ms/epoch - 1ms/step
Epoch 30/50
51/51 - 0s - loss: 1.2481 - mean_squared_error: 2.6988 - val_loss: 1.4293 - val_mean_squared_error: 3.5354 - 83ms/epoch - 2ms/step

TEST: [   0    3    5    7   24   27   33   39   42   47   52   55   60   62
   66   68   74   77   80   82   85   92   94   97  102  104  106  110
  117  125  132  136  137  144  165  183  204  213  222  227  228  232
  235  238  242  248  249  267  280  302  314  321  336  349  359  362
  364  388  404  424  434  446  448  467  523  525  533  536  541  542
  545  547  594  603  605  622  624  638  644  661  665  666  673  676
  688  691  716  718  737  762  770  793  796  803  811  826  842  846
  847  858  898  899  918  983  989 1026 1030 1041 1046 1049 1068 1074
 1079 1084 1101 1108 1111 1117 1120 1131 1134 1146 1161 1170 1179 1181
 1185 1189 1206 1208 1220 1223 1249 1258 1259 1283 1289 1293 1299 1324
 1326 1329 1335 1350 1361 1370 1374 1431 1440 1446 1447 1458 1468 1472
 1474 1477 1483 1492 1498 1501 1502 1503 1512 1517 1538 1542 1567 1571
 1575 1583 1601 1606 1615 1637 1662 1665 1666 1694 1706 1709 1723 1735
 1781 1786 1789 1799 1811 1812 1829 1860 1861 1885 1904 1909 1941 1958


Epoch 8/50
51/51 - 0s - loss: 2.2012 - mean_squared_error: 8.1729 - val_loss: 1.9637 - val_mean_squared_error: 5.5535 - 83ms/epoch - 2ms/step
Epoch 9/50
51/51 - 0s - loss: 2.0925 - mean_squared_error: 7.3308 - val_loss: 1.7914 - val_mean_squared_error: 4.8910 - 84ms/epoch - 2ms/step
Epoch 10/50
51/51 - 0s - loss: 1.8810 - mean_squared_error: 5.8813 - val_loss: 1.8159 - val_mean_squared_error: 5.0103 - 67ms/epoch - 1ms/step
Epoch 11/50
51/51 - 0s - loss: 1.6756 - mean_squared_error: 4.8806 - val_loss: 1.7904 - val_mean_squared_error: 4.8128 - 82ms/epoch - 2ms/step
Epoch 12/50
51/51 - 0s - loss: 1.7046 - mean_squared_error: 4.8641 - val_loss: 1.8233 - val_mean_squared_error: 5.0223 - 83ms/epoch - 2ms/step
Epoch 13/50
51/51 - 0s - loss: 1.6002 - mean_squared_error: 4.4493 - val_loss: 1.7528 - val_mean_squared_error: 4.4688 - 77ms/epoch - 2ms/step
Epoch 14/50
51/51 - 0s - loss: 1.6343 - mean_squared_error: 4.5233 - val_loss: 1.9455 - val_mean_squared_error: 5.5604 - 79ms/epoch - 2ms/step
E

Epoch 14/50
51/51 - 0s - loss: 1.7394 - mean_squared_error: 5.1299 - val_loss: 1.4899 - val_mean_squared_error: 3.1969 - 82ms/epoch - 2ms/step
Epoch 15/50
51/51 - 0s - loss: 1.5804 - mean_squared_error: 4.3520 - val_loss: 1.5174 - val_mean_squared_error: 3.3640 - 67ms/epoch - 1ms/step
Epoch 16/50
51/51 - 0s - loss: 1.5918 - mean_squared_error: 4.2689 - val_loss: 1.3729 - val_mean_squared_error: 3.0440 - 63ms/epoch - 1ms/step
Epoch 17/50
51/51 - 0s - loss: 1.5170 - mean_squared_error: 4.0439 - val_loss: 1.4520 - val_mean_squared_error: 3.2214 - 81ms/epoch - 2ms/step
Epoch 18/50
51/51 - 0s - loss: 1.4717 - mean_squared_error: 3.7365 - val_loss: 1.8156 - val_mean_squared_error: 4.8628 - 84ms/epoch - 2ms/step
Epoch 19/50
51/51 - 0s - loss: 1.7217 - mean_squared_error: 5.2357 - val_loss: 1.5545 - val_mean_squared_error: 3.7820 - 72ms/epoch - 1ms/step
Epoch 20/50
51/51 - 0s - loss: 1.5185 - mean_squared_error: 3.9506 - val_loss: 1.4345 - val_mean_squared_error: 3.4007 - 78ms/epoch - 2ms/step

Epoch 34/50
51/51 - 0s - loss: 1.2023 - mean_squared_error: 2.5621 - val_loss: 1.5893 - val_mean_squared_error: 3.7588 - 87ms/epoch - 2ms/step
Epoch 35/50
51/51 - 0s - loss: 1.2009 - mean_squared_error: 2.5945 - val_loss: 1.8011 - val_mean_squared_error: 4.7166 - 88ms/epoch - 2ms/step
Epoch 36/50
51/51 - 0s - loss: 1.1933 - mean_squared_error: 2.5593 - val_loss: 1.5336 - val_mean_squared_error: 3.9855 - 83ms/epoch - 2ms/step
Epoch 37/50
51/51 - 0s - loss: 1.2224 - mean_squared_error: 2.6525 - val_loss: 1.3882 - val_mean_squared_error: 3.0704 - 71ms/epoch - 1ms/step
Epoch 38/50
51/51 - 0s - loss: 1.1476 - mean_squared_error: 2.4036 - val_loss: 1.4207 - val_mean_squared_error: 3.4228 - 84ms/epoch - 2ms/step
Epoch 39/50
51/51 - 0s - loss: 1.1426 - mean_squared_error: 2.3638 - val_loss: 1.4713 - val_mean_squared_error: 3.4374 - 67ms/epoch - 1ms/step
TEST: [  13   14   20   21   34   40   64   87   91   95   98  121  130  134
  161  166  187  189  200  201  205  206  216  230  241  252  269

In [20]:
# sociodemographic predictors

y_dnn_socio_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

avg_r_squared = 0

def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

for train_index, test_index in ten_fold.split(X_sociodemo_test_passed):
    print("TEST:", test_index)

    X_train, X_test = X_sociodemo_test_passed.iloc[train_index], X_sociodemo_test_passed.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #scaler = StandardScaler()
    
    #scaled_X_train = scaler.fit_transform(X_train)
    #scaled_X_test = scaler.transform(X_test)
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    tf.random.set_seed(42)
    
    dnn_model = keras.models.Sequential([
        keras.layers.Dense(160,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(208,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(160,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(160,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(256,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(32,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(240,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(96,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(208,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(1)
    ])
    
    #early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    dnn_model.compile(optimizer="adam", loss=keras.losses.mean_squared_error, metrics=[keras.metrics.mean_squared_error])
    dnn_model.fit(x=scaled_X_train, y=y_train, epochs=50,   verbose=2) #callbacks=[early_stop],
    
    this_y_predict = dnn_model.predict(scaled_X_test).flatten()

    y_dnn_socio_predict = y_dnn_socio_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [  29   49   56   65   69   70   99  109  111  120  124  128  135  211
  212  231  233  237  239  251  254  256  270  281  298  305  306  307
  316  324  342  344  350  353  361  366  367  368  382  383  394  411
  414  416  420  422  427  429  433  450  464  480  483  485  486  514
  526  527  530  532  548  552  554  561  570  581  582  583  602  614
  617  630  679  741  743  752  756  759  785  798  819  824  843  849
  881  889  892  903  915  921  926  930  932  940  949  950  988  999
 1001 1009 1052 1087 1106 1113 1116 1118 1125 1137 1160 1169 1176 1177
 1182 1216 1221 1234 1235 1242 1244 1252 1255 1268 1270 1273 1274 1284
 1285 1287 1298 1303 1305 1316 1318 1322 1341 1343 1344 1357 1359 1379
 1384 1386 1412 1428 1429 1444 1448 1469 1476 1490 1497 1506 1531 1532
 1563 1573 1593 1594 1600 1604 1616 1619 1647 1659 1668 1671 1673 1675
 1681 1684 1686 1716 1720 1726 1732 1736 1740 1758 1764 1770 1784 1810
 1813 1814 1820 1855 1856 1867 1907 1918 1926 1934 1935 1936 1940 1952


Epoch 18/50
57/57 - 0s - loss: 2.9864 - mean_squared_error: 2.9864 - 111ms/epoch - 2ms/step
Epoch 19/50
57/57 - 0s - loss: 2.5146 - mean_squared_error: 2.5146 - 102ms/epoch - 2ms/step
Epoch 20/50
57/57 - 0s - loss: 2.4755 - mean_squared_error: 2.4755 - 126ms/epoch - 2ms/step
Epoch 21/50
57/57 - 0s - loss: 3.2252 - mean_squared_error: 3.2252 - 119ms/epoch - 2ms/step
Epoch 22/50
57/57 - 0s - loss: 2.7752 - mean_squared_error: 2.7752 - 112ms/epoch - 2ms/step
Epoch 23/50
57/57 - 0s - loss: 2.4041 - mean_squared_error: 2.4041 - 111ms/epoch - 2ms/step
Epoch 24/50
57/57 - 0s - loss: 2.6307 - mean_squared_error: 2.6307 - 106ms/epoch - 2ms/step
Epoch 25/50
57/57 - 0s - loss: 2.2687 - mean_squared_error: 2.2687 - 133ms/epoch - 2ms/step
Epoch 26/50
57/57 - 0s - loss: 2.3241 - mean_squared_error: 2.3241 - 121ms/epoch - 2ms/step
Epoch 27/50
57/57 - 0s - loss: 2.0662 - mean_squared_error: 2.0662 - 109ms/epoch - 2ms/step
Epoch 28/50
57/57 - 0s - loss: 2.0465 - mean_squared_error: 2.0465 - 133ms/epoch

Epoch 46/50
57/57 - 0s - loss: 1.3464 - mean_squared_error: 1.3464 - 114ms/epoch - 2ms/step
Epoch 47/50
57/57 - 0s - loss: 1.2224 - mean_squared_error: 1.2224 - 116ms/epoch - 2ms/step
Epoch 48/50
57/57 - 0s - loss: 1.2024 - mean_squared_error: 1.2024 - 133ms/epoch - 2ms/step
Epoch 49/50
57/57 - 0s - loss: 1.2189 - mean_squared_error: 1.2189 - 122ms/epoch - 2ms/step
Epoch 50/50
57/57 - 0s - loss: 1.5830 - mean_squared_error: 1.5830 - 116ms/epoch - 2ms/step
TEST: [   6   18   31   48   54   58   81   83   84   86  105  113  118  126
  147  155  174  177  179  181  208  214  221  236  243  277  285  286
  287  296  308  312  326  327  329  341  346  358  360  363  370  377
  380  423  428  430  435  442  451  457  461  468  490  493  500  505
  513  522  528  551  560  566  589  598  601  609  621  629  631  637
  643  660  669  670  677  694  700  701  721  722  746  755  764  771
  781  802  807  809  862  869  873  874  886  900  906  916  923  943
  952  968  974  978  994 1005 1006 1

Epoch 13/50
57/57 - 0s - loss: 3.4521 - mean_squared_error: 3.4521 - 116ms/epoch - 2ms/step
Epoch 14/50
57/57 - 0s - loss: 3.1215 - mean_squared_error: 3.1215 - 117ms/epoch - 2ms/step
Epoch 15/50
57/57 - 0s - loss: 3.8204 - mean_squared_error: 3.8204 - 122ms/epoch - 2ms/step
Epoch 16/50
57/57 - 0s - loss: 2.7904 - mean_squared_error: 2.7904 - 117ms/epoch - 2ms/step
Epoch 17/50
57/57 - 0s - loss: 2.7303 - mean_squared_error: 2.7303 - 117ms/epoch - 2ms/step
Epoch 18/50
57/57 - 0s - loss: 2.3835 - mean_squared_error: 2.3835 - 107ms/epoch - 2ms/step
Epoch 19/50
57/57 - 0s - loss: 2.6876 - mean_squared_error: 2.6876 - 124ms/epoch - 2ms/step
Epoch 20/50
57/57 - 0s - loss: 2.6389 - mean_squared_error: 2.6389 - 115ms/epoch - 2ms/step
Epoch 21/50
57/57 - 0s - loss: 2.3673 - mean_squared_error: 2.3673 - 90ms/epoch - 2ms/step
Epoch 22/50
57/57 - 0s - loss: 2.4813 - mean_squared_error: 2.4813 - 101ms/epoch - 2ms/step
Epoch 23/50
57/57 - 0s - loss: 2.3892 - mean_squared_error: 2.3892 - 100ms/epoch 

57/57 - 0s - loss: 1.9403 - mean_squared_error: 1.9403 - 135ms/epoch - 2ms/step
Epoch 42/50
57/57 - 0s - loss: 1.9343 - mean_squared_error: 1.9343 - 122ms/epoch - 2ms/step
Epoch 43/50
57/57 - 0s - loss: 1.7972 - mean_squared_error: 1.7972 - 116ms/epoch - 2ms/step
Epoch 44/50
57/57 - 0s - loss: 1.3091 - mean_squared_error: 1.3091 - 131ms/epoch - 2ms/step
Epoch 45/50
57/57 - 0s - loss: 1.2773 - mean_squared_error: 1.2773 - 128ms/epoch - 2ms/step
Epoch 46/50
57/57 - 0s - loss: 1.3910 - mean_squared_error: 1.3910 - 123ms/epoch - 2ms/step
Epoch 47/50
57/57 - 0s - loss: 1.2667 - mean_squared_error: 1.2667 - 109ms/epoch - 2ms/step
Epoch 48/50
57/57 - 0s - loss: 1.2227 - mean_squared_error: 1.2227 - 114ms/epoch - 2ms/step
Epoch 49/50
57/57 - 0s - loss: 1.6616 - mean_squared_error: 1.6616 - 118ms/epoch - 2ms/step
Epoch 50/50
57/57 - 0s - loss: 1.4135 - mean_squared_error: 1.4135 - 120ms/epoch - 2ms/step
TEST: [   9   11   28   36   38   57   61   75   79   89   90  108  114  133
  145  169  172

57/57 - 0s - loss: 5.5069 - mean_squared_error: 5.5069 - 97ms/epoch - 2ms/step
Epoch 9/50
57/57 - 0s - loss: 5.3313 - mean_squared_error: 5.3313 - 116ms/epoch - 2ms/step
Epoch 10/50
57/57 - 0s - loss: 4.2332 - mean_squared_error: 4.2332 - 116ms/epoch - 2ms/step
Epoch 11/50
57/57 - 0s - loss: 3.6849 - mean_squared_error: 3.6849 - 107ms/epoch - 2ms/step
Epoch 12/50
57/57 - 0s - loss: 4.2598 - mean_squared_error: 4.2598 - 104ms/epoch - 2ms/step
Epoch 13/50
57/57 - 0s - loss: 3.2453 - mean_squared_error: 3.2453 - 111ms/epoch - 2ms/step
Epoch 14/50
57/57 - 0s - loss: 3.5295 - mean_squared_error: 3.5295 - 100ms/epoch - 2ms/step
Epoch 15/50
57/57 - 0s - loss: 3.1884 - mean_squared_error: 3.1884 - 100ms/epoch - 2ms/step
Epoch 16/50
57/57 - 0s - loss: 2.7935 - mean_squared_error: 2.7935 - 92ms/epoch - 2ms/step
Epoch 17/50
57/57 - 0s - loss: 2.7326 - mean_squared_error: 2.7326 - 113ms/epoch - 2ms/step
Epoch 18/50
57/57 - 0s - loss: 2.5556 - mean_squared_error: 2.5556 - 96ms/epoch - 2ms/step
Epoc

Epoch 37/50
57/57 - 0s - loss: 1.7486 - mean_squared_error: 1.7486 - 116ms/epoch - 2ms/step
Epoch 38/50
57/57 - 0s - loss: 1.8197 - mean_squared_error: 1.8197 - 115ms/epoch - 2ms/step
Epoch 39/50
57/57 - 0s - loss: 1.4992 - mean_squared_error: 1.4992 - 131ms/epoch - 2ms/step
Epoch 40/50
57/57 - 0s - loss: 1.7172 - mean_squared_error: 1.7172 - 116ms/epoch - 2ms/step
Epoch 41/50
57/57 - 0s - loss: 1.4697 - mean_squared_error: 1.4697 - 138ms/epoch - 2ms/step
Epoch 42/50
57/57 - 0s - loss: 1.3798 - mean_squared_error: 1.3798 - 116ms/epoch - 2ms/step
Epoch 43/50
57/57 - 0s - loss: 1.8441 - mean_squared_error: 1.8441 - 131ms/epoch - 2ms/step
Epoch 44/50
57/57 - 0s - loss: 1.4951 - mean_squared_error: 1.4951 - 111ms/epoch - 2ms/step
Epoch 45/50
57/57 - 0s - loss: 1.6351 - mean_squared_error: 1.6351 - 119ms/epoch - 2ms/step
Epoch 46/50
57/57 - 0s - loss: 1.6306 - mean_squared_error: 1.6306 - 108ms/epoch - 2ms/step
Epoch 47/50
57/57 - 0s - loss: 1.7625 - mean_squared_error: 1.7625 - 131ms/epoch

In [4]:
dnn_socio_rmse = mean_squared_error(y_true , y_dnn_socio_predict, squared=False)
dnn_socio_r2 = r2_score(y_true, y_dnn_socio_predict)
avg_r_squared = avg_r_squared/10

In [5]:
# sociodemographic - 10 epochs
print("RMSE of the DNN model with sociodemographic predictors: "+str(dnn_socio_rmse))
print("R2 of the DNN model with sociodemographic predictors: "+str(dnn_socio_r2)) # for R2, I use this one.
print("R2 of the DNN model with sociodemographic predictors (avg): "+str(avg_r_squared))

RMSE of the DNN model with sociodemographic predictors: 2.462565140376982
R2 of the DNN model with sociodemographic predictors: 0.8249255081484549
R2 of the DNN model with sociodemographic predictors (avg): 0.8226863862891614


In [36]:
# Yingjie's run
dnn_socio_rmse = mean_squared_error(y_true , y_dnn_socio_predict, squared=False)
dnn_socio_r2 = r2_score(y_true, y_dnn_socio_predict)
avg_r_squared = avg_r_squared/10

In [37]:
# sociodemographic - 10 epochs
print("RMSE of the DNN model with sociodemographic predictors: "+str(dnn_socio_rmse))
print("R2 of the DNN model with sociodemographic predictors: "+str(dnn_socio_r2)) # for R2, I use this one.
print("R2 of the DNN model with sociodemographic predictors (avg): "+str(avg_r_squared))

RMSE of the DNN model with sociodemographic predictors: 2.044735055187158
R2 of the DNN model with sociodemographic predictors: 0.8792960437361126
R2 of the DNN model with sociodemographic predictors (avg): 0.8773809747331278
